In [ ]:
!pip install google-cloud-storage gcsfs

In [ ]:
pip install -U albumentations

In [ ]:
!pip install torch torchvision matplotlib segment-anything

In [ ]:
import os
import io
import random
from PIL import Image
import numpy as np
from google.cloud import storage
from segment_anything import SamPredictor, sam_model_registry
from sklearn.model_selection import train_test_split
import albumentations as A

In [7]:
!ls /kaggle/working/test_results

ls: cannot access '/kaggle/working/test_results': No such file or directory


In [15]:
try:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/kaggle/input/dronebucketfinalyear-json1/dronefinalyear-128e44efa683.json"
    client = storage.Client()
    print("Google Cloud Storage client initialized successfully.")
except Exception as e:
    print(f"Error initializing Google Cloud Storage client: {e}")

Google Cloud Storage client initialized successfully.


In [16]:
import os
from google.cloud import storage

# Replace 'your-service-account-file.json' with the exact filename of the JSON file in the directory
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/kaggle/input/dronebucketfinalyear-json1/dronefinalyear-128e44efa683.json"

# Initialize the client and connect to the bucket
try:
    client = storage.Client()
    bucket_name = 'drone_obstacle_detection'  # Replace with your bucket name
    bucket = client.bucket(bucket_name)
    print("Successfully connected to the Google Cloud Storage bucket.")
except Exception as e:
    print(f"Error initializing Google Cloud Storage client: {e}")


Successfully connected to the Google Cloud Storage bucket.


In [17]:
try:
    buckets = client.list_buckets()
    print("Buckets in project:")
    for bucket in buckets:
        print(bucket.name)
except Exception as e:
    print(f"Error listing buckets: {e}")


Buckets in project:
datanew2
drone_obstacle_detection
dronebucketfinalyear
dronebucketfinalyearpipeline
dronefinalyear_cloudbuild
gcf-v2-sources-775182290299-us-central1
gcf-v2-uploads-775182290299.us-central1.cloudfunctions.appspot.com
newbucket010


In [18]:
def check_gcs_connection(bucket_name, prefix):
    try:
        bucket = client.get_bucket(bucket_name)
        blobs = list(bucket.list_blobs(prefix=prefix))
        
        # Print the first few files to confirm connection
        print(f"Successfully connected to GCS bucket: '{bucket_name}' with prefix '{prefix}'")
        print("Sample files in the bucket:")
        for blob in blobs[:5]:  # Show a sample of the first 5 files
            print(f" - {blob.name}")
        print("\nGCS connection and access confirmed.\n")
    except Exception as e:
        print(f"Error accessing GCS bucket '{bucket_name}': {e}")

In [19]:
pip install google-cloud-storage

Note: you may need to restart the kernel to use updated packages.


In [20]:
# Define your bucket and prefix
bucket_name = 'drone_obstacle_detection'  # Replace with your actual GCS bucket name
input_prefix = 'drone_data/'       # Folder path in GCS for original images
output_prefix = 'Test-data_preprocessed/drone_data/'  # Folder path for preprocessed images
csv_output_prefix = 'Test-data_preprocessed/csv/'  # Folder for combined metadata CSV

In [21]:
# Run the connection check
check_gcs_connection(bucket_name, input_prefix)

Successfully connected to GCS bucket: 'drone_obstacle_detection' with prefix 'drone_data/'
Sample files in the bucket:
 - drone_data/train/neighbourhood/0/depth/0.png
 - drone_data/train/neighbourhood/0/depth/0_aug.png
 - drone_data/train/neighbourhood/0/depth/1.png
 - drone_data/train/neighbourhood/0/depth/10.png
 - drone_data/train/neighbourhood/0/depth/10_aug.png

GCS connection and access confirmed.



In [111]:
# Augmentation pipeline
def augment_image(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=45, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.GaussianBlur(p=0.3),
        A.GaussNoise(p=0.3)
    ])
    augmented = transform(image=np.array(image))
    return Image.fromarray(augmented["image"])

In [112]:
def augment_and_save_images(bucket_name, input_prefix):
    """
    Augments images in the specified folder structure and saves augmented versions.

    Args:
        bucket_name (str): Name of the GCS bucket.
        input_prefix (str): Prefix (path) in the bucket where images are stored.
    """
    # Initialize the GCS client and bucket
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # List all blobs (files) in the specified prefix
    blobs = list(bucket.list_blobs(prefix=input_prefix))
    for blob in blobs:
        if blob.name.endswith(('.png', '.jpg', '.jpeg')):
            # Load the image from GCS
            image_data = blob.download_as_bytes()
            image = Image.open(io.BytesIO(image_data)).convert("RGB")

            # Augment the image
            augmented_image = augment_image(image)

            # Create the augmented image path
            # Save the augmented image alongside the original with "_aug" suffix
            augmented_name = blob.name.replace(".jpg", "_aug.jpg").replace(".png", "_aug.png").replace(".jpeg", "_aug.jpeg")
            augmented_blob = bucket.blob(augmented_name)

            # Save augmented image to a buffer and upload to GCS
            buffer = io.BytesIO()
            augmented_image.save(buffer, format="PNG")
            buffer.seek(0)
            augmented_blob.upload_from_file(buffer, content_type="image/png")
            print(f"Saved augmented image to: {augmented_name}")

# Set your bucket name and prefix
bucket_name = "drone_obstacle_detection"

# Call the function to process and augment images
augment_and_save_images(bucket_name, input_prefix)

Saved augmented image to: drone_data/train/neighbourhood/0/depth/0_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/1_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/10_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/11_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/12_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/13_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/14_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/15_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/16_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/17_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/18_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/19_aug.png
Saved augmented image to: drone_data/train/neighbourhood/0/depth/2_aug.png
Saved augmented

In [113]:
# Depth Map Normalization
def normalize_depth(image):
    depth_array = np.array(image, dtype=np.float32)
    if depth_array.max() - depth_array.min() > 0:
        normalized = (depth_array - depth_array.min()) / (depth_array.max() - depth_array.min())
    else:
        normalized = depth_array
    return Image.fromarray((normalized * 255).astype(np.uint8))

In [114]:
# Flow Image Normalization
def normalize_flow(image):
    flow_array = np.array(image, dtype=np.float32)
    normalized = flow_array / 255.0
    return Image.fromarray((normalized * 255).astype(np.uint8))

In [115]:
# Standard Image Preprocessing
def preprocess_image(image):
    resized_image = image.resize((256, 256))
    normalized_array = np.array(resized_image) / 255.0
    return Image.fromarray((normalized_array * 255).astype(np.uint8))

In [116]:
# Segmentation Mask Processing
def preprocess_segmentation(image, class_mapping):
    mask_array = np.array(image)
    if mask_array.ndim == 3:
        mask_array = mask_array[:, :, 0]
    processed_mask = np.zeros_like(mask_array, dtype=np.uint8)
    for label, value in class_mapping.items():
        processed_mask[mask_array == label] = value
    return Image.fromarray(processed_mask, mode="L")

In [117]:
# Surface Normals Normalization
def normalize_surface_normals(image):
    normals_array = np.array(image, dtype=np.float32)
    normalized = (normals_array - normals_array.min()) / (normals_array.max() - normals_array.min())
    return Image.fromarray((normalized * 255).astype(np.uint8))

In [118]:
# Initialize Google Cloud Storage client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/kaggle/input/dronebucketfinalyear-json1/dronefinalyear-128e44efa683.json"
client = storage.Client()
bucket_name = 'drone_obstacle_detection'
input_prefix = 'drone_data/'  # Input folder path in GCS
output_prefix = 'Test-data_preprocessed/data'  # Output folder path in GCS
bucket = client.bucket(bucket_name)

In [119]:
def preprocess_and_save_images(bucket_name, input_prefix, output_prefix, class_mapping):
    """
    Preprocess images and save them in a different folder while maintaining the structure.

    Args:
        bucket_name (str): Name of the GCS bucket.
        input_prefix (str): Prefix (path) in the bucket to locate the images.
        output_prefix (str): Prefix (path) in the bucket to save preprocessed images.
        class_mapping (dict): Class mapping for segmentation preprocessing.
    """
    # Initialize the GCS client and bucket
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # List all blobs (files) in the specified prefix
    blobs = list(bucket.list_blobs(prefix=input_prefix))
    for blob in blobs:
        if blob.name.endswith(('.png', '.jpg', '.jpeg')):
            # Load the image from GCS
            image_data = blob.download_as_bytes()
            image = Image.open(io.BytesIO(image_data)).convert("RGB")

            # Preprocess based on the folder type
            if "depth" in blob.name:
                processed_image = normalize_depth(image)
            elif "flow" in blob.name:
                processed_image = normalize_flow(image)
            elif "segmentation" in blob.name:
                processed_image = preprocess_segmentation(image, class_mapping)
            elif "surfacenormals" in blob.name:
                processed_image = normalize_surface_normals(image)
            else:  # Standard RGB images
                processed_image = preprocess_image(image)

            # Save the preprocessed image in the output folder with the same structure
            processed_blob_name = blob.name.replace(input_prefix, output_prefix)
            processed_blob = bucket.blob(processed_blob_name)

            # Save preprocessed image to a buffer and upload to GCS
            buffer = io.BytesIO()
            processed_image.save(buffer, format="PNG")
            buffer.seek(0)
            processed_blob.upload_from_file(buffer, content_type="image/png")
            print(f"Saved preprocessed image to: {processed_blob_name}")

In [120]:
# Set your bucket name, input folder prefix, and output folder prefix
bucket_name = "drone_obstacle_detection"
input_prefix = "drone_data/"  # Input folder in GCS
output_prefix = "data_preprocessed/"  # Output folder in GCS
CLASS_MAPPING = {
    'ultra_thin': 255,
    'thin_structures': 240,
    'small_mesh': 220,
    'large_mesh': 200,
    'trees': 180,
    'buildings': 160,
    'vehicles': 140,
    'animals': 100,
    'other': 80
}

# Call the function to preprocess and save images
preprocess_and_save_images(bucket_name, input_prefix, output_prefix, CLASS_MAPPING)

Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/0.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/0_aug.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/1.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/10.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/10_aug.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/11.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/11_aug.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/12.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/12_aug.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/13.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/13_aug.png
Saved preprocessed image to: data_preprocessed/train/neighbourhood/0/depth/14.png

In [13]:
pip install autodistill autodistill-grounded-sam autodistill-yolov8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.0/527.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 25.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Set GCP credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/kaggle/input/dronebucketfinalyear-json1/dronefinalyear-128e44efa683.json"

In [16]:
# Initialize GCP storage client
client = storage.Client()
bucket_name = 'drone_obstacle_detection'  # Replace with your GCP bucket
bucket = client.bucket(bucket_name)

In [17]:
# Define paths in GCP
input_prefix = 'drone_data/'  # Folder in GCP for input images
output_prefix = 'dataset/output_data/'  # Folder for output images
annotations_prefix = 'dataset/annotations/'  # Folder for YOLO annotations
dataset_prefix = 'dataset/final_dataset/'  # Folder for the dataset
train_images_prefix = f"{dataset_prefix}train/images/"
train_labels_prefix = f"{dataset_prefix}train/labels/"
val_images_prefix = f"{dataset_prefix}val/images/"
val_labels_prefix = f"{dataset_prefix}val/labels/"
test_images_prefix = 'dataset/test_images/'  # Folder for test images
test_results_prefix = 'dataset/test_results/'  # Folder for test results
yaml_file_path = f"{dataset_prefix}dataset.yaml"  # YAML file path
annotated_train_images_prefix = 'dataset/annotated_images/train/'  # Folder for annotated training images
annotated_test_images_prefix = 'dataset/annotated_images/test/'  # Folder for annotated test images
plotted_annotations_prefix = 'dataset/plotted_annotations/'

In [126]:
# Create necessary directories in GCP
def create_gcs_folders(bucket, folder_paths):
    for path in folder_paths:
        blob = bucket.blob(path)
        blob.upload_from_string('', content_type='application/x-directory')

create_gcs_folders(bucket, [
    output_prefix, annotations_prefix, plotted_annotations_prefix, train_images_prefix, train_labels_prefix,
    val_images_prefix, val_labels_prefix, test_results_prefix,
    annotated_train_images_prefix, annotated_test_images_prefix
])

In [127]:
import os
import cv2
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
from autodistill.utils import plot

In [129]:
# Step 1: Annotate Images using GroundedSAM
ontology = CaptionOntology({
    "ultra_thin": "ultra-thin",
    "thin_structures": "thin",
    "poles": "poles",
    "fence": "fence",
    "wires": "wires",
    "mesh": "mesh",
    "trees": "trees",
    "forest": "forest",
    "bungalow": "bungalow",
    "car": "vehicles",
    "other": "miscellaneous"
})
base_model = GroundedSAM(ontology=ontology)

trying to load grounding dino directly
final text_encoder_type: bert-base-uncased


In [130]:
import matplotlib.pyplot as plt
import numpy as np

def convert_plot_to_numpy(plotted_image):
    """
    Converts a Matplotlib plot or similar object to a NumPy array compatible with OpenCV.
    """
    if isinstance(plotted_image, plt.Figure):
        # Render the Matplotlib figure to a NumPy array
        canvas = plotted_image.canvas
        canvas.draw()
        width, height = canvas.get_width_height()
        image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(height, width, 3)
        plt.close(plotted_image)  # Close the figure to free memory
        return image
    return plotted_image  # Already a valid NumPy array


In [132]:
def annotate_images(bucket, input_prefix, output_prefix, annotations_prefix):
    blobs = list(bucket.list_blobs(prefix=input_prefix))
    for blob in blobs:
        if blob.name.endswith(('.png', '.jpg', '.jpeg')):
            # Download image
            image_data = blob.download_as_bytes()
            image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
            if image is None:
                print(f"Failed to decode image: {blob.name}. Skipping...")
                continue
            img_height, img_width, _ = image.shape

            # Predict using GroundedSAM
            try:
                results = base_model.predict(image)
            except Exception as e:
                print(f"Error during prediction for {blob.name}: {e}")
                continue

            # Prepare annotations in YOLO format
            annotation_blob_name = blob.name.replace(input_prefix, annotations_prefix).replace('.jpg', '.txt').replace('.png', '.txt')
            annotation_blob = bucket.blob(annotation_blob_name)
            annotations = []

            for i in range(len(results.xyxy)):
                class_id = results.class_id[i]
                bbox = results.xyxy[i]
                x_min, y_min, x_max, y_max = bbox
                x_center = (x_min + x_max) / 2 / img_width
                y_center = (y_min + y_max) / 2 / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height
                annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

            try:
                annotation_blob.upload_from_string('\n'.join(annotations))
                print(f"Annotations uploaded to GCS: {annotation_blob_name}")
            except Exception as e:
                print(f"Error uploading annotations for {blob.name}: {e}")
                continue

            # Save images to the output folder
            output_blob_name = blob.name.replace(input_prefix, output_prefix)
            output_blob = bucket.blob(output_blob_name)

            try:
                _, buffer = cv2.imencode('.jpg', image)
                if buffer is None or len(buffer) == 0:
                    print(f"Failed to encode image for {blob.name}. Skipping...")
                    continue
                output_blob.upload_from_string(buffer.tobytes(), content_type='image/jpeg')
                print(f"Image uploaded to GCS: {output_blob_name}")
            except Exception as e:
                print(f"Error uploading image for {blob.name}: {e}")
                continue


annotate_images(bucket, input_prefix, output_prefix, annotations_prefix)


Annotations uploaded to GCS: dataset/annotations/train/neighbourhood/0/depth/0.txt
Image uploaded to GCS: dataset/output_data/train/neighbourhood/0/depth/0.png
Annotations uploaded to GCS: dataset/annotations/train/neighbourhood/0/depth/0_aug.txt
Image uploaded to GCS: dataset/output_data/train/neighbourhood/0/depth/0_aug.png
Annotations uploaded to GCS: dataset/annotations/train/neighbourhood/0/depth/1.txt
Image uploaded to GCS: dataset/output_data/train/neighbourhood/0/depth/1.png
Annotations uploaded to GCS: dataset/annotations/train/neighbourhood/0/depth/10.txt
Image uploaded to GCS: dataset/output_data/train/neighbourhood/0/depth/10.png
Annotations uploaded to GCS: dataset/annotations/train/neighbourhood/0/depth/10_aug.txt
Image uploaded to GCS: dataset/output_data/train/neighbourhood/0/depth/10_aug.png
Annotations uploaded to GCS: dataset/annotations/train/neighbourhood/0/depth/11.txt
Image uploaded to GCS: dataset/output_data/train/neighbourhood/0/depth/11.png
Annotations upload

In [ ]:
import random
import os

def split_dataset_flat_structure(
    bucket, 
    annotations_prefix, 
    output_prefix, 
    train_images_prefix, 
    train_labels_prefix, 
    val_images_prefix, 
    val_labels_prefix, 
    test_images_prefix
):
    """
    Splits the dataset into training, validation, and testing sets by copying the images and annotations
    to the respective folders, ensuring a flat folder structure for train, validation, and test sets.
    - 70% for training
    - 20% for validation
    - 10% for testing (no annotations for test set)
    """
    blobs = list(bucket.list_blobs(prefix=annotations_prefix))
    for blob in blobs:
        if blob.name.endswith('.txt'):
            # Get the corresponding image blob name (assuming .png images)
            image_blob_name = blob.name.replace(annotations_prefix, output_prefix).replace('.txt', '.png')
            
            # Generate flat file names by stripping folder paths
            annotation_filename = os.path.basename(blob.name)
            image_filename = os.path.basename(image_blob_name)
            
            # Randomly assign to train, validation, or testing
            rand = random.random()
            if rand < 0.7:  # 70% for training
                # Copy annotation file to training labels folder
                bucket.copy_blob(blob, bucket, new_name=os.path.join(train_labels_prefix, annotation_filename))
                # Copy image file to training images folder
                bucket.copy_blob(bucket.blob(image_blob_name), bucket, new_name=os.path.join(train_images_prefix, image_filename))
                print(f"Moved to training: {image_filename}, {annotation_filename}")
            elif rand < 0.9:  # 20% for validation
                # Copy annotation file to validation labels folder
                bucket.copy_blob(blob, bucket, new_name=os.path.join(val_labels_prefix, annotation_filename))
                # Copy image file to validation images folder
                bucket.copy_blob(bucket.blob(image_blob_name), bucket, new_name=os.path.join(val_images_prefix, image_filename))
                print(f"Moved to validation: {image_filename}, {annotation_filename}")
            else:  # 10% for testing (no labels for testing)
                # Only copy the image file to the test images folder
                bucket.copy_blob(bucket.blob(image_blob_name), bucket, new_name=os.path.join(test_images_prefix, image_filename))
                print(f"Moved to testing: {image_filename}")

# Call the function to split the dataset into train, validation, and test sets
split_dataset_flat_structure(
    bucket, 
    annotations_prefix, 
    output_prefix, 
    train_images_prefix, 
    train_labels_prefix, 
    val_images_prefix, 
    val_labels_prefix, 
    test_images_prefix
)


In [18]:
import yaml  # Import the yaml module
from google.cloud import storage

# Initialize the GCS client
client = storage.Client()

# Specify the GCS bucket name
bucket_name = "drone_obstacle_detection"
bucket = client.bucket(bucket_name)

# Define the dataset prefix (path in the bucket)
dataset_prefix = "dataset/final_dataset/"

# Define your dataset_yaml structure
dataset_yaml = {
    "path": "gs://drone_obstacle_detection/dataset/final_dataset",
    'train': "train/images",
    'val': "val/images",
    'test': "test/images",  # Test images path
    'nc': 11,  # Number of classes
    'names': [
        "ultra-thin", "thin", "poles", "fence", "wires", 
        "mesh", "trees", "forest", "bungalow", "vehicles", "miscellaneous"
    ]
}

# YAML file path (with .yaml extension explicitly included)
yaml_file_path = 'dataset/dataset.yaml'

# Upload the YAML file to GCS
yaml_blob = bucket.blob(yaml_file_path)
yaml_blob.upload_from_string(yaml.dump(dataset_yaml), content_type="application/x-yaml")

print(f"Uploaded dataset YAML to GCS at {yaml_file_path}")


Uploaded dataset YAML to GCS at dataset/dataset.yaml


In [152]:
pip install google-cloud-storage


Note: you may need to restart the kernel to use updated packages.


In [153]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [154]:
import os
import cv2
import shutil
import yaml
from ultralytics import YOLO
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
import matplotlib.pyplot as plt

In [155]:
from google.cloud import storage

# Authenticate if needed
client = storage.Client()

# Set bucket name and path to dataset.yaml
bucket_name = 'drone_obstacle_detection'
file_path = 'dataset/dataset.yaml'

# Get the bucket
bucket = client.get_bucket(bucket_name)

# Check if file exists
blob = bucket.blob(file_path)
if blob.exists():
    print(f"File {file_path} exists in the bucket.")
else:
    print(f"File {file_path} does not exist.")


File dataset/dataset.yaml exists in the bucket.


In [118]:
x = 'gs://drone_obstacle_detection/dataset/final_dataset/dataset.yaml'

In [157]:
!gsutil ls gs://drone_obstacle_detection/dataset/

gs://drone_obstacle_detection/dataset/dataset.yaml
gs://drone_obstacle_detection/dataset/annotated_images/
gs://drone_obstacle_detection/dataset/annotations/
gs://drone_obstacle_detection/dataset/final_dataset/
gs://drone_obstacle_detection/dataset/output_data/
gs://drone_obstacle_detection/dataset/plotted_annotations/
gs://drone_obstacle_detection/dataset/test_images/
gs://drone_obstacle_detection/dataset/test_results/
